In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
s = '''from collections import defaultdict
def dd():
    return defaultdict(float)
'''
with open('utils.py','w') as f:
    f.write(s)

from utils import dd
import pandas as pd
import numpy as np
import gc
from sklearn.metrics import roc_auc_score
from collections import defaultdict
from tqdm.notebook import tqdm
import lightgbm as lgb
import riiideducation
import matplotlib.pyplot as plt
import seaborn as sns

import random
import os

from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import cpu_count
import pickle as pk
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import os
import bisect

In [ ]:
def questions_and_lecture_parsing():
    q = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv').fillna('')
    bundle_dict = q.groupby('bundle_id')['question_id'].count().to_dict()
    q.loc[:, 'small_part'] = -1
    cnt = 0
    for part in range(1, 8):
        end = q[q.part == part][q[q.part == part]['question_id'].diff(-1) != -1]['question_id'].values
        start = q[q.part == part][q[q.part == part]['question_id'].diff() != 1]['question_id'].values
        for s,t in zip(start, end):
            q.loc[s:t, 'small_part'] = cnt
            cnt += 1

    q['part'] -= 1
    
    for i in range(3):
        q[f'tags{i}'] = q['tags'].apply(lambda x: [int(item) for item in x.split()][i] if len(x.split()) > i else np.nan)
    for i in range(1,3):
        q[f'tags{i}'] = q[f'tags{i-1}']*200+q[f'tags{i}']
        
    q['tags_count'] = q['tags'].apply(lambda x: len(x.split()))  
    return q

def mdiv(a, b):
    if b == 0:
        return np.nan
    return a*1./b

def ll(predicted, actual, eps=1e-14):
    predicted = np.clip(predicted, eps, 1-eps)
    loss = -1*(actual * np.log(predicted) + (1 - actual) * np.log(1-predicted))
    return loss

# Random seed
SEED = 42

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    
seed_everything(SEED)

In [ ]:
# Funcion for user stats with loops
def add_features(df, 
        last_u_content_id_dict,
        last_u_container_id_dict,
        hist_u_answered_correctly_cnt_dict,
        hist_u_elapsed_time_sum_dict,
        hist_u_explanation_sum_dict,
        hist_u_same_part_correctly_cnt_dict,
        hist_u_same_content_id_correctly_cnt_dict,
        timestamp_u,
        hist_u_answered_correctly_sum_dict,
        hist_u_score_sum_dict,
        hist_u_same_part_correctly_sum_dict,
        hist_u_same_content_id_correctly_sum_dict,
        last_u_last_incorrect_timestamp_dict,
        hist_u_last_incorrect_cnt_dict,
        hist_u_lag_time_sum_dict,
        global_avg_q_time_dict, prior_question_elapsed_time_mean, global_content_cnt_dict,
        update = True):
  
    # -----------------------------------------------------------------------
    last_u_diff_container_id = np.zeros(len(df), dtype = np.float32)
    timestamp_u_recency_1 = np.zeros(len(df), dtype = np.float32) 
    timestamp_u_gap_time_ratio = np.zeros(len(df), dtype = np.float32) 
    timestamp_u_avg_time_ratio = np.zeros(len(df), dtype = np.float32) 
    timestamp_u_lag_time_ratio = np.zeros(len(df), dtype = np.float32) 
    timestamp_u_lag_time = np.zeros(len(df), dtype = np.float32)

    hist_u_lag_time_raito = np.zeros(len(df), dtype = np.float32) 
    hist_u_answered_correctly_ratio = np.zeros(len(df), dtype = np.float32) 
    hist_u_elapsed_time_ratio = np.zeros(len(df), dtype = np.float32) 
    hist_u_explanation_ratio = np.zeros(len(df), dtype = np.float32)
    hist_u_score_ratio = np.zeros(len(df), dtype = np.float32) 
    hist_u_same_part_sum = np.zeros(len(df), dtype = np.float32)
    hist_u_same_part_cnt = np.zeros(len(df), dtype = np.float32)
    hist_u_same_part_ratio = np.zeros(len(df), dtype = np.float32)
    hist_u_same_content_id_sum = np.zeros(len(df), dtype = np.float32)
    hist_u_same_content_id_cnt = np.zeros(len(df), dtype = np.float32)
    hist_u_same_content_id_ratio = np.zeros(len(df), dtype = np.float32)
    hist_u_last_incorrect_timestamp = np.zeros(len(df), dtype = np.float32)
    hist_u_last_incorrect_cnt = np.zeros(len(df), dtype = np.float32)
    # User Question
    answered_correctly_uq_count = np.zeros(len(df), dtype = np.int32)
    # -----------------------------------------------------------------------
    
    
    for num, row in tqdm(enumerate(df[['user_id','answered_correctly','content_id','prior_question_elapsed_time', 
                                  'prior_question_had_explanation', 'timestamp','contentid_mean','task_container_id','part']].values)):
                
        last_u_content_id = last_u_content_id_dict.get(row[0], np.nan)
        last_u_container_id = last_u_container_id_dict.get(row[0], np.nan)
        
        last_u_diff_container_id[num] = row[7] - last_u_container_id   # 1

        last_u_sum_time = row[3] * global_content_cnt_dict.get(last_u_content_id, 1)
        sum_time_consum = global_avg_q_time_dict.get(row[2], prior_question_elapsed_time_mean) * global_content_cnt_dict.get(row[2], 1)
        
        # Client features assignation
        # ------------------------------------------------------------------
            
        if len(timestamp_u[row[0]]) == 0:
            timestamp_u_recency_1[num] = np.nan     # 2
            timestamp_u_gap_time_ratio[num] = np.nan  # 3
            timestamp_u_avg_time_ratio[num] = np.nan  # 4
            timestamp_u_lag_time_ratio[num] = np.nan  # 5
            timestamp_u_lag_time[num]  = np.nan # 6
            
        elif len(timestamp_u[row[0]]) == 1:
            timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][0]
            timestamp_u_gap_time_ratio[num] = mdiv(sum_time_consum , (row[5] - timestamp_u[row[0]][0]))
            timestamp_u_avg_time_ratio[num] = mdiv((row[5] - timestamp_u[row[0]][0]), global_content_cnt_dict.get(row[2], 1))
            timestamp_u_lag_time_ratio[num] = np.nan
            timestamp_u_lag_time[num]  = np.nan
            
        elif len(timestamp_u[row[0]]) == 2:
            timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][1]
            timestamp_u_gap_time_ratio[num] = mdiv(sum_time_consum , (row[5] - timestamp_u[row[0]][1]))
            timestamp_u_avg_time_ratio[num] = mdiv((row[5] - timestamp_u[row[0]][1]) , global_content_cnt_dict.get(row[2], 1))
            timestamp_u_lag_time_ratio[num] = mdiv((timestamp_u[row[0]][1] - timestamp_u[row[0]][0] - last_u_sum_time) , (row[5] - timestamp_u[row[0]][1]+1))
            timestamp_u_lag_time[num] = (timestamp_u[row[0]][1] - timestamp_u[row[0]][0] - last_u_sum_time)
            
        elif len(timestamp_u[row[0]]) == 3:
            timestamp_u_recency_1[num] = row[5] - timestamp_u[row[0]][2]
            timestamp_u_gap_time_ratio[num] = mdiv(sum_time_consum , (row[5] - timestamp_u[row[0]][2]))
            timestamp_u_avg_time_ratio[num] = mdiv((row[5] - timestamp_u[row[0]][2]) , global_content_cnt_dict.get(row[2], 1))
            timestamp_u_lag_time_ratio[num] = mdiv((timestamp_u[row[0]][2] - timestamp_u[row[0]][1] - sum_time_consum) , (row[5] - timestamp_u[row[0]][2]+1))
            timestamp_u_lag_time[num] = (timestamp_u[row[0]][2] - timestamp_u[row[0]][1] - last_u_sum_time)

        if timestamp_u_lag_time[num] is not np.nan:    
            hist_u_lag_time_sum_dict[row[0]] += max(0, min(timestamp_u_lag_time[num], 300*1000))
        hist_u_lag_time_raito[num] = mdiv(hist_u_lag_time_sum_dict[row[0]], hist_u_answered_correctly_cnt_dict[row[0]])

                
        hist_u_answered_correctly_ratio[num] = mdiv(hist_u_answered_correctly_sum_dict[row[0]], hist_u_answered_correctly_cnt_dict[row[0]])
        hist_u_elapsed_time_ratio[num] = mdiv(hist_u_elapsed_time_sum_dict[row[0]], hist_u_answered_correctly_cnt_dict[row[0]])
        hist_u_explanation_ratio[num] = mdiv(hist_u_explanation_sum_dict[row[0]], hist_u_answered_correctly_cnt_dict[row[0]])
            
        hist_u_score_ratio[num] = mdiv(hist_u_score_sum_dict[row[0]], hist_u_answered_correctly_cnt_dict[row[0]])
        
        hist_u_same_part_sum[num] = hist_u_same_part_correctly_sum_dict[row[0]][row[8]]
        hist_u_same_part_cnt[num] = hist_u_same_part_correctly_cnt_dict[row[0]][row[8]]
        hist_u_same_part_ratio[num] = mdiv(hist_u_same_part_sum[num], hist_u_same_part_cnt[num])

        hist_u_same_content_id_sum[num] = hist_u_same_content_id_correctly_sum_dict[row[0]][row[2]]
        hist_u_same_content_id_cnt[num] = hist_u_same_content_id_correctly_cnt_dict[row[0]][row[2]]        
        hist_u_same_content_id_ratio[num] = mdiv(hist_u_same_content_id_sum[num], hist_u_same_content_id_cnt[num])
    
        hist_u_last_incorrect_timestamp[num] = row[5] - last_u_last_incorrect_timestamp_dict[row[0]]
        hist_u_last_incorrect_cnt[num] = hist_u_last_incorrect_cnt_dict[row[0]]
        
        
        # ------------------------------------------------------------------
        # Client features updates
        hist_u_answered_correctly_cnt_dict[row[0]] += 1
        hist_u_elapsed_time_sum_dict[row[0]] += row[3]
        hist_u_explanation_sum_dict[row[0]] += int(row[4])
        hist_u_same_part_correctly_cnt_dict[row[0]][row[8]] +=1
        hist_u_same_content_id_correctly_cnt_dict[row[0]][row[2]] += 1

        if len(timestamp_u[row[0]])==0 or row[5]!=timestamp_u[row[0]][-1]:
            last_u_content_id_dict[row[0]] = row[2]
            if len(timestamp_u[row[0]]) == 3:
                timestamp_u[row[0]].pop(0)
                timestamp_u[row[0]].append(row[5])
            else:
                timestamp_u[row[0]].append(row[5])
        if len(timestamp_u[row[0]])!=0 or row[5]==timestamp_u[row[0]][-1]:
            last_u_container_id_dict[row[0]] = row[7]
        
        # Flag for training and inference
        if update:
            # ------------------------------------------------------------------
            # Client features updates
            hist_u_answered_correctly_sum_dict[row[0]] += row[1]
            hist_u_score_sum_dict[row[0]] += (1 if row[1] == 1 else -1) * ll(row[6], row[1])
            hist_u_same_part_correctly_sum_dict[row[0]][row[8]] += row[1]
            hist_u_same_content_id_correctly_sum_dict[row[0]][row[2]] += row[1]
            if row[1] == 0:
                last_u_last_incorrect_timestamp_dict[row[0]] = row[5]
                hist_u_last_incorrect_cnt_dict[row[0]] = 0
            else:
                hist_u_last_incorrect_cnt_dict[row[0]] += 1

    user_df = pd.DataFrame({
                            'last_u_diff_container_id': last_u_diff_container_id, 
                            'timestamp_u_recency_1': timestamp_u_recency_1, 
                            'timestamp_u_gap_time_ratio': timestamp_u_gap_time_ratio, 
                            'timestamp_u_avg_time_ratio': timestamp_u_avg_time_ratio, 
                            'timestamp_u_lag_time_ratio': timestamp_u_lag_time_ratio, 
                            'timestamp_u_lag_time': timestamp_u_lag_time, 
        
                            'hist_u_lag_time_raito': hist_u_lag_time_raito,
                            'hist_u_answered_correctly_ratio': hist_u_answered_correctly_ratio, 
                            'hist_u_elapsed_time_ratio': hist_u_elapsed_time_ratio, 
                            'hist_u_explanation_ratio': hist_u_explanation_ratio,
                            'hist_u_score_ratio': hist_u_score_ratio, 
                            'hist_u_same_part_sum': hist_u_same_part_sum,
                            'hist_u_same_part_cnt': hist_u_same_part_cnt,
                            'hist_u_same_part_ratio': hist_u_same_part_ratio,
                            'hist_u_same_content_id_sum': hist_u_same_content_id_sum,
                            'hist_u_same_content_id_cnt':hist_u_same_content_id_cnt,
                            'hist_u_same_content_id_ratio':hist_u_same_content_id_ratio,
                            'hist_u_last_incorrect_timestamp':hist_u_last_incorrect_timestamp,
                            'hist_u_last_incorrect_cnt':hist_u_last_incorrect_cnt,
        
                           })
    
    df = pd.concat([df.reset_index(drop=True), user_df], axis = 1)
    return df


def update_features(df, 
                    hist_u_answered_correctly_sum_dict, 
                    hist_u_score_sum_dict, 
                    hist_u_same_part_correctly_sum_dict, 
                    hist_u_same_content_id_correctly_sum_dict,
                    last_u_last_incorrect_timestamp_dict,
                    hist_u_last_incorrect_cnt_dict):
    for row in df[['user_id','answered_correctly','content_id','prior_question_elapsed_time', 
                    'prior_question_had_explanation', 'timestamp','contentid_mean',
                   'task_container_id','part','content_type_id']].values:
        if row[-1] == 0:
            # ------------------------------------------------------------------
            # Client features updates
            hist_u_answered_correctly_sum_dict[row[0]] += row[1]
            hist_u_score_sum_dict[row[0]] += (1 if row[1] == 1 else -1) * ll(row[6], row[1])
            hist_u_same_part_correctly_sum_dict[row[0]][row[8]] += row[1]
            hist_u_same_content_id_correctly_sum_dict[row[0]][row[2]] += row[1]
            if row[1] == 0:
                last_u_last_incorrect_timestamp_dict[row[0]] = row[5]
                hist_u_last_incorrect_cnt_dict[row[0]] = 0
            else:
                hist_u_last_incorrect_cnt_dict[row[0]] += 1
            
    return

In [ ]:
def inference(TARGET, FEATURES, model, questions_df, prior_question_elapsed_time_mean, features_dicts, global_dicts):
    
    # Get feature dict
    last_u_content_id_dict = features_dicts['last_u_content_id_dict']
    last_u_container_id_dict = features_dicts['last_u_container_id_dict']
    hist_u_answered_correctly_cnt_dict = features_dicts['hist_u_answered_correctly_cnt_dict']
    hist_u_elapsed_time_sum_dict = features_dicts['hist_u_elapsed_time_sum_dict']
    hist_u_explanation_sum_dict = features_dicts['hist_u_explanation_sum_dict']
    hist_u_same_part_correctly_cnt_dict = features_dicts['hist_u_same_part_correctly_cnt_dict']
    hist_u_same_content_id_correctly_cnt_dict = features_dicts['hist_u_same_content_id_correctly_cnt_dict']
    timestamp_u = features_dicts['timestamp_u']
    hist_u_answered_correctly_sum_dict = features_dicts['hist_u_answered_correctly_sum_dict']
    hist_u_score_sum_dict = features_dicts['hist_u_score_sum_dict']
    hist_u_same_part_correctly_sum_dict = features_dicts['hist_u_same_part_correctly_sum_dict']
    hist_u_same_content_id_correctly_sum_dict = features_dicts['hist_u_same_content_id_correctly_sum_dict']
    last_u_last_incorrect_timestamp_dict = features_dicts['last_u_last_incorrect_timestamp_dict']
    hist_u_last_incorrect_cnt_dict = features_dicts['hist_u_last_incorrect_cnt_dict']
    hist_u_lag_time_sum_dict = features_dicts['hist_u_lag_time_sum_dict']
    
    # Get global dict
    global_avg_q_time_dict = global_dicts['global_avg_q_time_dict']
    prior_question_elapsed_time_mean = global_dicts['prior_question_elapsed_time_mean']
    global_content_cnt_dict = global_dicts['global_content_cnt_dict']
    contentidmean_dict = global_dicts['contentidmean_dict']
    
    # Get api iterator and predictor
    env = riiideducation.make_env()
    iter_test = env.iter_test()
    set_predict = env.predict
    
    previous_test_df = None
    for (test_df, sample_prediction_df) in iter_test:
        if previous_test_df is not None:
            previous_test_df[TARGET] = eval(test_df["prior_group_answers_correct"].iloc[0])
            previous_test_df = pd.merge(previous_test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
            previous_test_df['contentid_mean'] = previous_test_df['content_id'].map(lambda x:contentidmean_dict.get(x,np.nan))
            update_features(previous_test_df,
                    hist_u_answered_correctly_sum_dict, 
                    hist_u_score_sum_dict, 
                    hist_u_same_part_correctly_sum_dict, 
                    hist_u_same_content_id_correctly_sum_dict,
                    last_u_last_incorrect_timestamp_dict,
                    hist_u_last_incorrect_cnt_dict)
                            
        previous_test_df = test_df.copy()
        test_df = test_df[test_df['content_type_id'] == 0].reset_index(drop = True)
        test_df['prior_question_had_explanation'] = test_df.prior_question_had_explanation.fillna(False).astype('int8')
        test_df['prior_question_elapsed_time'].fillna(prior_question_elapsed_time_mean, inplace = True)
        test_df = pd.merge(test_df, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
        test_df[TARGET] = 0
        test_df['contentid_mean'] = test_df['content_id'].map(lambda x:contentidmean_dict.get(x,np.nan))
        test_df = add_features(test_df,
        last_u_content_id_dict,
        last_u_container_id_dict,
        hist_u_answered_correctly_cnt_dict,
        hist_u_elapsed_time_sum_dict,
        hist_u_explanation_sum_dict,
        hist_u_same_part_correctly_cnt_dict,
        hist_u_same_content_id_correctly_cnt_dict,
        timestamp_u,
        hist_u_answered_correctly_sum_dict,
        hist_u_score_sum_dict,
        hist_u_same_part_correctly_sum_dict,
        hist_u_same_content_id_correctly_sum_dict,
        last_u_last_incorrect_timestamp_dict,
        hist_u_last_incorrect_cnt_dict,
        hist_u_lag_time_sum_dict,
        global_avg_q_time_dict, prior_question_elapsed_time_mean, global_content_cnt_dict,
        update = False)
        test_df[TARGET] =  model.predict(test_df[FEATURES])
        set_predict(test_df[['row_id', TARGET]])
        
    print('Job Done')
 

In [ ]:
META_PATHS = '../input/meta-files/meta.test.pkl'
TARGET, FEATURES, model, questions_df, prior_question_elapsed_time_mean, features_dicts, global_dicts = pd.read_pickle(META_PATHS)
inference(TARGET, FEATURES, model, questions_df, prior_question_elapsed_time_mean, features_dicts, global_dicts)